In [1]:
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import os
import shutil
from sklearn.model_selection import train_test_split

In [2]:
df=pd.read_csv('trainLabels.csv')
df.head()

,image,level
0,10_left,0
1,10_right,0
2,13_left,0
3,13_right,0
4,15_left,1


In [3]:
base_dir='base_dir'
os.mkdir(base_dir)

train_dir=os.path.join(base_dir,'train_dir')
os.mkdir(train_dir)

val_dir=os.path.join(base_dir,'val_dir')
os.mkdir(val_dir)

From the csv file we get to know that there are 5 levels of DR casesthus we are making 5 different directories to store images of each cases or levels

In [6]:
Normal=os.path.join(train_dir,'0')
os.mkdir(Normal)
Mild_NPDR=os.path.join(train_dir,'1')
os.mkdir(Mild_NPDR)
Moderate_NPDR=os.path.join(train_dir,'2')
os.mkdir(Moderate_NPDR)
Severe_NPDR=os.path.join(train_dir,'3')
os.mkdir(Severe_NPDR)
PDR=os.path.join(train_dir,'4')
os.mkdir(PDR)

In [7]:
Normal=os.path.join(val_dir,'0')
os.mkdir(Normal)
Mild_NPDR=os.path.join(val_dir,'1')
os.mkdir(Mild_NPDR)
Moderate_NPDR=os.path.join(val_dir,'2')
os.mkdir(Moderate_NPDR)
Severe_NPDR=os.path.join(val_dir,'3')
os.mkdir(Severe_NPDR)
PDR=os.path.join(val_dir,'4')
os.mkdir(PDR)

In [9]:
y=df['level']

In [10]:
df_train,df_val=train_test_split(df,test_size=0.1,random_state=101,stratify=y)

In [11]:
print(df_train.shape)
print(df_val.shape)

(31597, 2)
(3511, 2)


In [12]:
df_train['level'].value_counts()

0    23222
2     4759
1     2194
3      785
4      637
Name: level, dtype: int64

In [13]:
df_val['level'].value_counts()

0    2580
2     529
1     244
3      87
4      71
Name: level, dtype: int64

In [14]:
df.set_index('image',inplace=True)

In [15]:
fol_1=os.listdir('./eyepacs_preprocess')

In [16]:
TRAIN_LIST=list(df_train['image'])
VAL_LIST=list(df_val['image'])

In [17]:
for image in TRAIN_LIST:
    fname=image+'.jpeg'
    labels=df.loc[image,'level']
    
    if fname in fol_1:
        #we are making src and dst files to copy from our dataset into seaprate directories
        src=os.path.join('./eyepacs_preprocess',fname)
        dst=os.path.join(train_dir,str(labels),fname)
        shutil.copyfile(src,dst)

In [18]:
for image in VAL_LIST:
    fname=image+'.jpeg'
    labels=df.loc[image,'level']
    
    if fname in fol_1:
        #we are making src and dst files to copy from our dataset into seaprate directories
        src=os.path.join('./eyepacs_preprocess',fname)
        dst=os.path.join(val_dir,str(labels),fname)
        shutil.copyfile(src,dst)

In [19]:
# This should be same as df_train['level'].value_counts()

print(len(os.listdir('base_dir/_dir/train_dir/0')))
print(len(os.listdir('base_dir/train_dir/1')))
print(len(os.listdir('base_dir/train_dir/2')))
print(len(os.listdir('base_dir/train_dir/3')))
print(len(os.listdir('base_dir/train_dir/4')))


print(len(os.listdir('base_dir/val_dir/0')))
print(len(os.listdir('base_dir/val_dir/1')))
print(len(os.listdir('base_dir/val_dir/2')))
print(len(os.listdir('base_dir/val_dir/3')))
print(len(os.listdir('base_dir/val_dir/4')))

23222
2194
4759
785
637
2580
244
529
87
71


### Data Augmentation

In [22]:
classes=['0','1','2','3','4']
for i in classes:
    aug_dir='aug_dir'
    os.mkdir(aug_dir)
    
    img_dir=os.path.join(aug_dir,'img_dir')
    os.mkdir(img_dir)
    
    img_class=i
    img_list=os.listdir('base_dir/train_dir/'+img_class)
    
    #copying images from train_dir to img_dir
    for fname in img_list:
        src=os.path.join('base_dir/train_dir/'+img_class,fname)
        dst=os.path.join(img_dir,fname)
        shutil.copyfile(src,dst)
    
    #pointing to a dir containing images and not directly the images
    path=aug_dir
    save_path='base_dir/train_dir/'+img_class
    
    datagen=ImageDataGenerator(
        rotation_range=180,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest'
    )
    batch_size=50
    aug_datagen=datagen.flow_from_directory(path,
                                           save_to_dir=save_path,
                                           save_format='jpeg',
                                           target_size=(224,224),
                                            batch_size=batch_size)
    # We will limit the augmented images to 1000 per class/level
    num_aug_wanted=1000
    num_files=len(os.listdir(img_dir))
    num_batches=int(np.ceil((num_aug_wanted-num_files)/batch_size))
    
    for i in range(0,num_batches):
        imgs,label=next(aug_datagen)
    
    #removing the temporary directory
    shutil.rmtree('aug_dir')

Found 23222 images belonging to 1 classes.
Found 2194 images belonging to 1 classes.
Found 4759 images belonging to 1 classes.
Found 785 images belonging to 1 classes.
Found 637 images belonging to 1 classes.


In [23]:
print(len(os.listdir('base_dir/train_dir/0')))
print(len(os.listdir('base_dir/train_dir/1')))
print(len(os.listdir('base_dir/train_dir/2')))
print(len(os.listdir('base_dir/train_dir/3')))
print(len(os.listdir('base_dir/train_dir/4')))

23222
2194
4759
1035
1037


## Preprocessing done using ImageDataGenerator